# Homework 2: Word Similarity

Student Name:

Student ID:

Python version used:

## General info

<b>Due date</b>: 1pm, Sunday April 1st

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 5% of mark for class

<b>Overview</b>: In this homework, you'll be quantifying the similarity between pairs of words using the structure of WordNet and word co-occurrence in the Brown corpus, using PMI, LSA, and word2vec. You will quantify how well these methods work by comparing to a carefully filtered human annotated gold-standard.

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> It is recommended to use Python 2 but we accept Python 3 solutions</b>. Make sure you state which version you used in the beggining of this notebook.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Extra credit</b>: Each homework has a task which is optional with respect to getting full marks on the assignment, but that can be used to offset any points lost on this or any other homework assignment (but not the final project or the exam). We recommend you skip over this step on your first pass, and come back if you have time: the amount of effort required to receive full marks (1 point) on an extra credit question will be substantially more than earning the same amount of credit on other parts of the homework.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.


<b>Instructions</b>: For this homework we will be comparing our methods against a popular dataset of word similarities called Similarity-353. You need to first obtain this data set, which can be downloaded <a href="http://www.cs.technion.ac.il/~gabr/resources/data/wordsim353/wordsim353.zip">here</a>. The file we will be using is called *combined.tab*. Except for the header (which should be stripped out), the file is tab formated with the first two columns corresponding to two words, and the third column representing a human-annotated similarity between the two words.

Assume the file *combined.tab* is located <b>in the same folder as this notebook</b>. You should load this file into a Python dictionary (NOTE: in Python, tuples of strings, i.e. ("tiger","cat") can serve as the keys of dictionaries). This dataset contains many rare words: we need to filter this dataset in order for it to be better suited to the resources we will use in this assignment. So your first goal is to filter this dataset to generate a smaller test set where you will evaluate your word similarity methods.

The first filtering is based on document frequencies in the Brown corpus, in order to remove rare words. In this assignment, we will be treating the <i>paragraphs</i> of the Brown corpus as our "documents", you can iterate over them by using the `paras` method of the corpus reader. You should start by creating a Python list where each element of the list is a set containing the word <b>types</b> from a different paragraph of the Brown corpus: the words should be lower-cased and lemmatized before they are added to the set (keep it around, because you will need this list again later on). Then, using the information in this corpus, calculate document frequencies and remove from your test set any word pairs where at least one of the two words has a document frequency of less than 10 in this corpus. 

The second filtering is based on words with highly ambiguous senses and involves using the NLTK interface to WordNet. Here, you should remove any words which do not have a *single primary sense*. We define single primary sense here as either having only one sense (i.e. only one synset), or where the count (as provided by the WordNet `count()` method for the lemmas associated with a synset) of the most common sense is at least five and at least five times larger than the next most common sense. Also, you should remove any words where the primary sense is not a noun (this information is also in the synset). Store the synset corresponding to this primary sense in a dictionary for use in the next section. Given this definition, remove any word pairs from the test set where at least one of the words does not contain a single primary sense or if the single primary sense is not a noun.

When you have applied these two filtering steps, print out all the pairs in your filtered test set (if you have done this correctly, the total should be more than 10, but less than 50).

(1.5 marks)

In [7]:
import nltk

nltk.download('wordnet')
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
def lemmatize(word):
    ###return the lemma form of input token###
    ###first lemmatize as verb###
    ###if does not change, lemmatize as noun#
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

sim353_dict = {}
sim353_read = open("combined.tab").readlines()
sim353_read.pop(0)

for line in sim353_read:
    ele = line.split()
    if len(ele) == 3:
        word1 = lemmatize(ele[0].lower())
        word2 = lemmatize(ele[1].lower())
        sim353_dict[(word1, word2)] = float(ele[2])


brownset = []
nltk.download('brown')
from nltk.corpus import brown
for paragraph in brown.paras():
    para = paragraph[0]
    processed = []
    for word in para:
        word = lemmatize(word.lower())
        processed.append(word)
    brownset.append(set(processed))

def item_freq((word1, word2), min = 10, doc = brownset):
    ###count document frequency of a word pair
    counter1 = 0
    counter2 = 0
    for item in doc:
        if counter1 >= min and counter2 >= min:
            return True
        else:
            if counter1 < min:
                if word1 in item:
                    counter1 += 1
            if counter2 < min:
                if word2 in item:
                    counter2 += 1
    return False


###remove less frequent word pairs
for word_pair in sim353_dict.keys():
    if not item_freq(word_pair):
        del sim353_dict[word_pair]



nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_primary(word):
    word_synset = wn.synsets(word)

    if len(word_synset) == 1 and word_synset[0].pos() == 'n':
        return word_synset[0]
    if len(word_synset) > 1:
        syn_count = {}
        for syn in word_synset:
            lemmas = syn.lemmas()
            for lem in lemmas:
                lem_name = lem.name().lower()
                if word == lem_name:
                    syn_count[syn] = lem.count()
                    break
        count_list = syn_count.values()
        count_list.sort(reverse=True)
        if count_list[0] >= 5 and count_list[0] >= 5*count_list[1]:
            for syn in syn_count.keys():
                if count_list[0] == syn_count[syn] and syn.pos() == 'n':
                    return syn
    return None

###filter out word without primary meaning
word_synsets = {}
for word_pair in sim353_dict.keys():
    (word1, word2) = word_pair
    synset1 = get_primary(word1)
    synset2 = get_primary(word2)
    if synset1 and synset2:
        word_synsets[word1] = synset1
        word_synsets[word2] = synset2
    else:
        del sim353_dict[word_pair]
#print len(sim353_dict.keys())
print sim353_dict.keys()

[nltk_data] Downloading package wordnet to /Users/Rena/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to /Users/Rena/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Rena/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[('planet', 'moon'), ('school', 'center'), ('doctor', 'personnel'), ('type', 'kind'), ('planet', 'people'), ('coast', 'forest'), ('car', 'automobile'), ('coast', 'hill'), ('coast', 'shore'), ('baby', 'mother'), ('century', 'year'), ('stock', 'egg'), ('professor', 'doctor')]


<b>Instructions</b>: Now you will create several dictionaries with similarity scores for pairs of words in your test set derived using the techniques discussed in class. The first of these is the Wu-Palmer scores derived from the hypernym relationships in WordNet, which you should calculate using the primary sense for each word derived above. You can use the built-in method included in the NLTK interface, you don't have to implement your own. When you're done,  print out the Python dictionary of word pair/similarity mappings. 

(0.5 marks)

In [8]:
wup_sim = {}
for (word1, word2) in sim353_dict.keys():
    synset1 = word_synsets[word1]
    synset2 = word_synsets[word2]
    wup_sim[(word1, word2)] = synset1.wup_similarity(synset2)
print "wup_sim", wup_sim

wup_sim {('stock', 'egg'): 0.11764705882352941, ('coast', 'hill'): 0.6666666666666666, ('coast', 'shore'): 0.9090909090909091, ('baby', 'mother'): 0.5, ('professor', 'doctor'): 0.5, ('coast', 'forest'): 0.16666666666666666, ('car', 'automobile'): 1.0, ('type', 'kind'): 0.9473684210526315, ('planet', 'moon'): 0.8, ('school', 'center'): 0.13333333333333333, ('century', 'year'): 0.8333333333333334, ('planet', 'people'): 0.18181818181818182, ('doctor', 'personnel'): 0.13333333333333333}


**Instructions:** Next, you will calculate Positive PMI (PPMI) for your word pairs using statistics derived from the Brown: you should use the same set up as you did to calculate document frequency above: paragraphs as documents, lemmatized, lower-cased, and with term frequency information removed by conversion to Python sets. You need to use the basic method for calculating PPMI introduced in class (and also in the reading) which is appropriate for any possible definition of co-occurrence (here, appearing in the same paragraph), but you should only calculate PPMI for the words in your test set. You must avoid building the entire co-occurrence matrix, instead you should keeping track of the sums you need for the probabilities as you go along. When you have calculated PMI for all the pairs, your code should print out the Python dictionary of word-pair/PPMI-similarity mappings.

(1 mark)

In [9]:
import math

def get_PMI((word1, word2), docset):
    word1_count = 0
    word2_count = 0
    both_count = 0
    total_doc = 0.0
    for para in docset:
        total_doc += len(para)
        if word1 in para:
            word1_count += 1
            if word2 in para:
                word2_count += 1
                both_count += 1
        elif word2 in para:
            word2_count += 1
        else:
            continue
    if both_count <= 0:
        return 0
    else:
        result = math.log((both_count/total_doc)/((word1_count/total_doc)*(word2_count/total_doc)))
    return max(result, 0)

ppmi_dict = {}
for word_pair in sim353_dict.keys():
    ppmi_dict[word_pair] = get_PMI(word_pair, brownset)

print "ppmi_sim", ppmi_dict

ppmi_sim {('stock', 'egg'): 0, ('coast', 'hill'): 0, ('coast', 'shore'): 0, ('baby', 'mother'): 5.013267061873542, ('professor', 'doctor'): 0, ('coast', 'forest'): 0, ('car', 'automobile'): 5.204509873937463, ('type', 'kind'): 3.6466273126888558, ('planet', 'moon'): 8.209020469320889, ('school', 'center'): 3.7221915104633445, ('century', 'year'): 3.041890306381168, ('planet', 'people'): 0, ('doctor', 'personnel'): 5.642698428047379}


**Instructions:** Next, you will derive similarity scores using the LSA method, i.e. apply SVD and truncate to get a dense vector and then use cosine similarity between the two vectors for each word pair. You can use the Distributed Semantics notebook as a starting point, but note that since you are interested here in word semantics, you will be constructing a matrix where the (non-sparse) rows correspond to words in the vocabulary, and the (sparse) columns correspond to the texts where they appear (this is the opposite of the notebook). Again, use the Brown corpus, in the same format as with PMI and document frequency. After you have a matrix in the correct format, use truncatedSVD in Sci-kit learn to produce dense vectors of length 500, and then use cosine similarity to produce similarities for your word pairs. Print out the corresponding Python dictionary.

(1 mark)

In [10]:
def get_word_matrix(docset, target_word, word_list):
    ###matrix row: target words from word pair + vocabulary
    ###matrix column: if word appear in doc
    ###word_lookup: keep track of the index for the target words
    word_freq_matrix = []
    word_lookup = {}
    lookup_counter = 0
    for word in target_word:
    	word_freq = get_word_freq(word, docset)
        word_freq_matrix.append(word_freq)
        word_lookup[word] = lookup_counter
        lookup_counter += 1
    for word in word_list:
    	word_freq = get_word_freq(word, docset)
    	word_freq_matrix.append(word_freq)
    return word_lookup, word_freq_matrix



def get_word_freq(word, docset):
    ###step through the document###
    ###check if word exist in the doc###
    ###return dictionary with doc:if_appear###
	word_freq = []
	dic = {}
	counter = 0
	for para in docset:
		if word in para:
			dic[counter] = 1
		else:
			dic[counter] = 0
		counter += 1
	return dic

def get_vocabulary(wordset, doc, size = 500):
    ###returned vocabulary contains both target words###
    ###and the tokens in the doc###
	doc_voca = []
	counter = size - len(wordset)
	for para in doc:
		for token in para:
			if token not in word and counter > 0:
				doc_voca.append(token)
				counter -= 1
	return doc_voca


from sklearn.feature_extraction import DictVectorizer
vectorizer = DictVectorizer()

target_word = set(i for sub in sim353_dict.keys() for i in sub)
vocabulary = get_vocabulary(target_word, brownset)


lookup_dict, word_matrix = get_word_matrix(brownset, target_word, vocabulary)
word_matrix = vectorizer.fit_transform(word_matrix)

from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components = 500)
truncated_matrix = svd.fit_transform(word_matrix)


from sklearn.metrics.pairwise import cosine_similarity as cosine
cos_sim = {}
for word_pair in sim353_dict.keys():
    (word1, word2) = word_pair
    index1 = lookup_dict[word1]
    index2 = lookup_dict[word2]
    result = cosine([truncated_matrix[index1]], [truncated_matrix[index2]])
    cos_sim[word_pair] = result[0][0]
print "cos_sim", cos_sim

cos_sim {('stock', 'egg'): 4.803015624110785e-16, ('coast', 'hill'): -6.305719835175694e-16, ('coast', 'shore'): 3.95516952522712e-16, ('baby', 'mother'): 0.023414645289544123, ('professor', 'doctor'): 2.1684043449710089e-16, ('coast', 'forest'): -2.1267709815475655e-15, ('car', 'automobile'): 0.036435977886793834, ('type', 'kind'): 0.011822941265747094, ('planet', 'moon'): 0.20044593143431907, ('school', 'center'): 0.021266436150248293, ('century', 'year'): 0.015134538658618801, ('planet', 'people'): -3.625572064791527e-16, ('doctor', 'personnel'): 0.03207501495497848}


**Instructions:** Next, you will derive a similarity score from word2vec vectors, using the Gensim interface. Check the Gensim word2vec tutorial for details on the API: https://radimrehurek.com/gensim/models/word2vec.html. Again, you should use the Brown for this, but for word2vec you don't need to worry about paragraphs: feel free to train your model at the sentence level instead. Your vectors should have the same number of dimensions as LSA (500), and you need to run for 50 iterations. This may take a while (several minutes), but that's okay, you won't be marked based on the speed of this. You should extract the similarites you need directly from the Gensim model, put them in a Python dictionary, and print them out.

(0.5 mark)

In [13]:
brown_sent = brown.sents()
from gensim.models import Word2Vec
w2v_model = Word2Vec(brown_sent, size = 500, iter = 50)

w2v_sim = {}
for word_pair in sim353_dict.keys():
    (word1, word2) = word_pair
    w2v_sim[word_pair] = w2v_model.wv.similarity(word1, word2)
print "w2v_sim", w2v_sim

w2v_sim {('stock', 'egg'): 0.16338709293335052, ('coast', 'hill'): 0.43194400335338307, ('coast', 'shore'): 0.44624137855437973, ('baby', 'mother'): 0.22203210978283336, ('professor', 'doctor'): 0.09946103281283186, ('coast', 'forest'): 0.2914740875661272, ('car', 'automobile'): 0.16285589338322337, ('type', 'kind'): 0.2585200376374964, ('planet', 'moon'): 0.1572712298477399, ('school', 'center'): -0.02426203873186266, ('century', 'year'): 0.3075953016108216, ('planet', 'people'): -0.0033652778316725293, ('doctor', 'personnel'): -0.04937328483102168}



**Instructions:** Finally, you should compare all the similarities you've created to the gold standard you loaded and filtered in the first step. For this, you can use the Pearson correlation co-efficient (`pearsonr`), which is included in scipy (`scipy.stats`). Be careful converting your dictionaries to lists for this purpose, the data for the two datasets needs to be in the same order for correct comparison using correlation. Write a general function, then apply it to each of the similarity score dictionaries, and print out the result for each (be sure to label them!). Hint: All of the methods used here should be markedly above 0, but also far from 1 (perfect correlation); if you're not getting reasonable results, go back and check your code for bugs!  

(0.5 mark)


In [14]:
from scipy.stats import pearsonr

def get_pearsonr(sim1, sim2 = sim353_dict):
    sim1_list = []
    sim2_list = []
    for word_pair in sim2.keys():
        sim1_list.append(sim1[word_pair])
        sim2_list.append(sim2[word_pair])
    (co_eff, p_val) = pearsonr(sim1_list, sim2_list)
    return co_eff

print "sim353 - wup", get_pearsonr(wup_sim)
print "sim353 - ppmi", get_pearsonr(ppmi_dict)
print "sim353 - cos_sim", get_pearsonr(cos_sim)
print "sim353 - w2v_sim", get_pearsonr(w2v_sim)

sim353 - wup 0.8519061523656611
sim353 - ppmi 0.41738004480472835
sim353 - cos_sim 0.2937942293184316
sim353 - w2v_sim 0.2768353363076464


## Bonus: Improving the correlation

**Instructions:** To get the extra credit, you should try to get a noticeably (at least 0.05) higher correlation than that of the highest correlation that you found in the main assignment. To accomplish this, you can use any resources included or accessible via NLTK EXCEPT the Google word2vec vectors. Half of your mark here will be based on the thought and effort involved in your solution (if you tried something sensible that didn't work, show us), the other half will be based on the actual performance increase, which you should demonstrate. For full credit, be sure to explain exactly what you are doing.

(1 bonus mark) 

In [ ]:
###two methods are tried to improve the performance
###and the main focus is to preprocess the corpus
###1st: to remove the stop words and punctuation before feeding the brownset to the estimator
###Result: the pearsonr decreases for PPMI

###2nd: to use word sense match instead of exact character match
###Trial1: simply trying to match the character in the lemmas from the word sense
###Result: the pearsonr decreases for PPMI
###Observation: cases like "shore", "coast", "coast" has lemma such as "seashore", but exact match will filter it out
###Trial2: use Regex to search in the lemma
###Result: sim353 - new_ppmi_sim 0.508531355227 comparing to previous 0.417380044805

In [15]:
###1st Method
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords = set(stopwords.words('english'))

def process_brown(docset, word_filter):
    new_brown = []
    for para in brownset:
        new_para = []
        for token in para:
            if token.isalpha() and token not in word_filter:
                new_para.append(token)
        new_brown.append(new_para)
    return new_brown

processed_brown = process_brown(brownset, stopwords)

new_ppmi_dict = {}
for word_pair in sim353_dict.keys():
    new_ppmi_dict[word_pair] = get_PMI(word_pair, processed_brown)

print "ppmi_sim", new_ppmi_dict
print "sim353 - new_ppmi_sim", get_pearsonr(new_ppmi_dict)



###2nd Method
def get_para_lemma(paragraph):
    para_lem = []
    for token in paragraph:
        lem = [lemma.name().lower() for syn in wn.synsets(token) for lemma in syn.lemmas()]
        for l in lem:
            para_lem.append(l)
    return para_lem

import re
def find_word(word, word_list):
    pattern = re.compile(word)
    for w in word_list:
        if pattern.search(w):
            return True
    return False


def get_PMI_new((word1, word2), docset):
    word1_count = 0
    word2_count = 0
    both_count = 0
    total_doc = 0.0

    for para in docset:
        total_doc += len(para)
        para_lemma = get_para_lemma(para)
        if find_word(word1, para_lemma):
            word1_count += 1
            if find_word(word2, para_lemma):
                word2_count += 1
                both_count += 1
        elif find_word(word2, para_lemma):
            word2_count += 1
        else:
            continue
    #total_doc = float(len(brownset))
    if both_count <= 0:
        return 0
    else:
        result = math.log((both_count/total_doc)/((word1_count/total_doc)*(word2_count/total_doc)))
    #print word1_count, word2_count, both_count, total_doc
    return max(result, 0)



ppmi_dict = {}
for word_pair in sim353_dict.keys():
    ppmi_dict[word_pair] = get_PMI_new(word_pair, brownset)

print "ppmi_sim", ppmi_dict
print "sim353 - new_ppmi_sim", get_pearsonr(ppmi_dict)

[nltk_data] Downloading package stopwords to /Users/Rena/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
ppmi_sim {('stock', 'egg'): 0, ('coast', 'hill'): 0, ('coast', 'shore'): 0, ('baby', 'mother'): 4.327387780145035, ('professor', 'doctor'): 0, ('coast', 'forest'): 0, ('car', 'automobile'): 4.518630592208955, ('type', 'kind'): 2.960748030960348, ('planet', 'moon'): 7.523141187592381, ('school', 'center'): 3.0363122287348374, ('century', 'year'): 2.356011024652661, ('planet', 'people'): 0, ('doctor', 'personnel'): 4.956819146318872}
sim353 - new_ppmi_sim 0.41068871041585087
ppmi_sim {('stock', 'egg'): 2.9024021916790503, ('coast', 'hill'): 0, ('coast', 'shore'): 6.613135523291492, ('baby', 'mother'): 3.629080682549932, ('professor', 'doctor'): 3.304738246795211, ('coast', 'forest'): 3.610659125262128, ('car', 'automobile'): 3.977533944486599, ('type', 'kind'): 3.249348140357088, ('planet', 'moon'): 3.647866148341929, ('school', 'center'): 3.0870579399111207, ('cen

## A final word

Normally, we would not use a corpus as small as the Brown for the purposes of building distributional word vectors. Also, note that filtering our test set to just words we are likely to do well on would typically be considered cheating.